In [5]:
import os
import openai
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display


In [10]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)


# Check the key

if not client.api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API key found and looks good so far!")


# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


# Let's try one out. Change the website and add print statements to follow along.

input_website = Website("https://learn.microsoft.com/en-us/answers/")
print(input_website.title)
print(input_website.text)

# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown format. \
If it includes news or announcements, then summarize these too in markdown format.\n\n"
    user_prompt += website.text
    return user_prompt

user_prompt_for(input_website)

print(user_prompt_for(input_website))

# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# Try this out, and then try for a few more websites

messages_for(input_website)

# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content


#summarize("http://apple.com")


# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://learn.microsoft.com/en-us/answers/")

API key found and looks good so far!
Microsoft Q&A | Microsoft Learn
Skip to main content
This browser is no longer supported.
Upgrade to Microsoft Edge to take advantage of the latest features, security updates, and technical support.
Download Microsoft Edge
More info about Internet Explorer and Microsoft Edge
Table of contents
Welcome to Microsoft Q&A
What do you need help with today?
Find it on Q&A — the home for technical questions and answers at Microsoft. New to Q&A? See our get started article below.
Supported products
.NET
Microsoft Technologies based on the .NET software framework.
Azure
A cloud computing platform and infrastructure for building, deploying and managing applications and services through a worldwide network of Microsoft-managed datacenters.
HoloLens
A family of Microsoft self-contained, holographic devices that enable engagement with digital content and interaction with holograms in the surrounding environment.
Microsoft 365
Formerly Office 365, is a line of sub

# Summary of Microsoft Q&A Website
=====================================

Microsoft Q&A is a platform for technical questions and answers at Microsoft. It provides a wide range of topics, including:

### Supported Products
The website lists various Microsoft-supported products, including:
* .NET
* Azure
* HoloLens
* Microsoft 365
* Microsoft Advertising
* Microsoft Graph
* Microsoft Intune
* Microsoft Partner Center
* Microsoft BizTalk Server
* Microsoft System Center
* Microsoft Teams
* Microsoft Viva
* Office
* SQL Server
* Sysinternals
* Visual Studio
* Windows

### How to Use Q&A
The website also provides guides on how to use the platform, including:

* **Get started**: Learn about the features and how to find solutions
* **Writing a quality question**: Tips for asking clear, specific, and well-structured questions
* **Writing a quality answer**: Guidance on responding to questions with detailed, relevant, and respectful answers

### Other Resources
In addition to Q&A, the website offers:

* Microsoft Learn community: A space to share learning experiences and connect with experts
* Learning rooms: Find connections with experts and peers facilitated by Microsoft Learn experts
* Dynamics: A collection of user groups for Dynamics 365 business applications
* Credentials Support forums: Ask questions related to Microsoft certifications

# Announcements
### Release Notes
The website includes release notes and enhancements for the Q&A platform.